# Quantizing a model with ONNX Runtime for text classification tasks

This notebook shows how to apply different post-training quantization approaches such as static and dynamic quantization using [ONNX Runtime](https://onnxruntime.ai), for any tasks of the GLUE benchmark. This is made possible thanks to 🤗 [Optimum](https://github.com/huggingface/optimum), an extension of 🤗 [Transformers](https://github.com/huggingface/transformers), providing a set of performance optimization tools enabling maximum efficiency to train and run models on targeted hardwares.  

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers, 🤗 Datasets and 🤗 Optimum. Uncomment the following cell and run it.

In [ ]:
#! pip install datasets transformers[sklearn] optimum[onnxruntime]

Make sure your version of 🤗 Optimum is at least 1.1.0 since the functionality was introduced in that version:

In [18]:
from optimum.version import __version__

print(__version__)

1.4.0.dev0


The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. This dataset is built from the SQuAD dataset.
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. This dataset is built from the Winograd Schema Challenge dataset.

We will see how to apply post-training static quantization on a DistilBERT model fine-tuned on the SST-2 task:

In [8]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
task = "sst2"
model_checkpoint = "textattack/bert-base-uncased-SST-2"

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the dataset and get the metric we need to use for evaluation. This can be easily done with the functions `load_dataset` and `load_metric`.  

In [9]:
from datasets import load_dataset, load_metric

`load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
validation_split = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
eval_dataset = load_dataset("glue", actual_task, split=validation_split)
metric = load_metric("glue", actual_task) 

Note that `load_metric` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

To preprocess our dataset, we will need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [13]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer than what the model selected can handle will be truncated to the maximum length accepted by the model.

In [14]:
sentence1_key, sentence2_key = task_to_keys[task]

def preprocess_function(examples, tokenizer):
    args = (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
    return tokenizer(*args, padding="max_length", max_length=128, truncation=True)

## Applying quantization on the model

We can set our `quantization_approach` to either `dynamic` or `static` in order to apply respectively dynamic and static quantization. 
- Post-training static quantization : introduces an additional calibration step where data is fed through the network in order to compute the activations quantization parameters.
- Post-training dynamic quantization : dynamically computes activations quantization parameters based on the data observed at runtime. 

In [15]:
QUANTIZATION_APPROACH = ["dynamic", "static"]

quantization_approach = "static"

First, let's create the output directory where the resulting quantized model will be saved.

In [27]:
import os

model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-{quantization_approach}-quantization"
os.makedirs(output_dir, exist_ok=True)

We will use the [🤗 Optimum](https://github.com/huggingface/optimum) library to instantiate an `ORTQuantizer`, which will take care of the quantization process. To instantiate an `ORTQuantizer`, we need to provide a path to a converted ONNX checkpoint or instance of a `ORTModelForXXX`.

In [ ]:
from optimum.onnxruntime import ORTQuantizer, ORTModelForSequenceClassification

# Loading Model from the Hub and convert to ONNX
ort_model = ORTModelForSequenceClassification.from_pretrained(model_checkpoint, from_transformers=True)

# Create a quantizer from a ORTModelForXXX
quantizer = ORTQuantizer.from_pretrained(ort_model)


We also need to create an `QuantizationConfig` instance, which is the configuration handling the ONNX Runtime quantization related parameters.

* We set `per_channel` to `False` in order to apply per-tensor quantization on the weights. As opposed to per-channel quantization, which introduces one set of quantization parameters per channel, per-tensor quantization means that there will be one set of quantization parameters per tensor.
* We set the number of samples `num_calibration_samples` to use for the calibration step resulting from static quantization to `40`.
* `operators_to_quantize` is used to specify the types of operators to quantize, here we want to quantize all the network's fully connected and embedding layers.

In [22]:
from optimum.onnxruntime.configuration import QuantizationConfig, AutoCalibrationConfig
from optimum.onnxruntime.quantization import QuantFormat, QuantizationMode, QuantType

per_channel = False
num_calibration_samples = 40
operators_to_quantize = ["MatMul", "Add", "Gather"] 
apply_static_quantization = quantization_approach == "static"

qconfig = QuantizationConfig(
    is_static=apply_static_quantization,
    format=QuantFormat.QDQ if apply_static_quantization else QuantFormat.QOperator,
    mode=QuantizationMode.QLinearOps if apply_static_quantization else QuantizationMode.IntegerOps,
    activations_dtype=QuantType.QInt8 if apply_static_quantization else QuantType.QUInt8,
    weights_dtype=QuantType.QInt8,
    per_channel=per_channel,
    operators_to_quantize=operators_to_quantize,
)

When applying static quantization, we need to perform a calibration step where the activations quantization ranges are computed. This additionnal step should only be performed in the case of static quantization and not for dynamic quantization. 
Because the quantization of certain nodes often results in degradation in accuracy, we create an instance of `QuantizationPreprocessor` to determine the nodes to exclude when applying static quantization.

In [ ]:
from functools import partial
from transformers import AutoTokenizer
from optimum.onnxruntime.preprocessors import QuantizationPreprocessor
from optimum.onnxruntime.preprocessors.passes import (
    ExcludeGeLUNodes,
    ExcludeLayerNormNodes,
    ExcludeNodeAfter,
    ExcludeNodeFollowedBy,
)

# Load tokenizer for preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

ranges = None 
quantization_preprocessor = None
if apply_static_quantization:
    # Create the calibration dataset used for the calibration step
    calibration_dataset = quantizer.get_calibration_dataset(
        "glue",
        dataset_config_name=actual_task,
        preprocess_function=partial(preprocess_function, tokenizer=tokenizer),
        num_samples=num_calibration_samples,
        dataset_split="train",
    )
    calibration_config = AutoCalibrationConfig.minmax(calibration_dataset)
    # Perform the calibration step: computes the activations quantization ranges
    ranges = quantizer.fit(
        dataset=calibration_dataset,
        calibration_config=calibration_config,
        operators_to_quantize=qconfig.operators_to_quantize,
    )
    quantization_preprocessor = QuantizationPreprocessor()
    # Exclude the nodes constituting LayerNorm
    quantization_preprocessor.register_pass(ExcludeLayerNormNodes())
    # Exclude the nodes constituting GELU
    quantization_preprocessor.register_pass(ExcludeGeLUNodes())
    # Exclude the residual connection Add nodes
    quantization_preprocessor.register_pass(ExcludeNodeAfter("Add", "Add"))
    # Exclude the Add nodes following the Gather operator
    quantization_preprocessor.register_pass(ExcludeNodeAfter("Gather", "Add"))
    # Exclude the Add nodes followed by the Softmax operator
    quantization_preprocessor.register_pass(ExcludeNodeFollowedBy("Add", "Softmax"))

Finally, we export the quantized model.

In [ ]:
quantizer.quantize(
    save_dir=output_dir,
    calibration_tensors_range=ranges,
    quantization_config=qconfig,
    preprocessor=quantization_preprocessor,
)

## Evaluation

To evaluate our resulting quantized model we need to define how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B).

The metric chosen to evaluate the quantized model's performance will be Matthews correlation coefficient (MCC) for CoLA, Pearson correlation coefficient (PCC) for STS-B and accuracy for any other tasks.

In [29]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Then to apply the preprocessing on all the sentences (or pairs of sentences) of our validation dataset, we just use the `map` method of our `dataset` object that was earlier created. This will apply the `preprocess_function` function on all the elements of our validation dataset.

In [30]:
eval_dataset = eval_dataset.map(partial(preprocess_function, tokenizer=tokenizer), batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

Finally, to estimate the drop in performance resulting from quantization, we are going to perform an evaluation step for both models (before and after quantization). In order to perform the latter, we will need to instantiate an `ORTModel` and thus need:

* The path of the model to evaluate.
* The dataset to use for the evaluation step.
* The model's ONNX configuration `onnx_config` associated to the model. This instance of  `OnnxConfig` describes how to export the model through the ONNX format.
* The function that will be used to compute the evaluation metrics `compute_metrics` that was defined previously.

In [32]:
from optimum.onnxruntime import ORTModel
from pathlib import Path

ort_model = ORTModel(quantizer.onnx_model_path, compute_metrics=compute_metrics, label_names=["label"])
model_output = ort_model.evaluation_loop(eval_dataset)

In [33]:
model_output.metrics

{'accuracy': 0.9243119266055045}

In [36]:
q8_ort_model = ORTModel(Path(output_dir) / "model_quantized.onnx", compute_metrics=compute_metrics, label_names=["label"])
q8_model_output = q8_ort_model.evaluation_loop(eval_dataset)

In [37]:
q8_model_output.metrics

{'accuracy': 0.9002293577981652}

Now let's compute the full-precision and the quantized model respective size in megabyte (MB) :

In [40]:
fp_model_size = os.path.getsize(quantizer.onnx_model_path) / (1024*1024)
q_model_size = os.path.getsize(Path(output_dir) / "model_quantized.onnx") / (1024*1024)

The reduction in the model size resulting from quantization is:

In [41]:
round(fp_model_size / q_model_size, 2)

3.92